# Installs & Imports
Just press play.

In [14]:
# Install required python packages
%%capture
!pip install oauth2client
!pip install google-api-python-client
!pip install httplib2
!pip install great_tables polars
!pip install python-dateutil
!pip install jupyter_ui_poll
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2

In [15]:
# OAuth2 Authentication and API Integration
from oauth2client.client import OAuth2WebServerFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import httplib2

# Data Handling and Processing
import json
import requests
import polars as pl
import pandas as pd

# Date and Time Manipulation
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import time

# Table Validation and Schema
import great_tables as gt
from great_tables import GT, md

# File Management
from google.colab import files
import csv

# UI and Widgets
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
from IPython.display import display, Javascript
from time import sleep

# Google Colab Secrets
from google.colab import userdata

# Ad
from IPython.display import display, HTML

# Authentication
*Requires User Input*

1. Press play & use the drop down to display the hidden cells.
2. An authentication URL will be added to the output panel (second cell block down). Click this.
3. Login using the Google Account that has access to the GSC Property you're trying to extract data from.
4. Once logged in, Google will generate an authentication code for you, You can copy this via the copy button next to the authentication code.
5. Paste this code in the next output panel that requests this code.
6. Press enter.

Reminder: Each code block needs to be played in order.

In [16]:
# This section handles the secure retrieval of DataForSEO API credentials from Google Colab's
# secrets management system.
#
# Prerequisites:
# 1. You must have a Google API account
# 2. Store your credentials in Colab's secrets manager:
#    - Click the key icon in the left sidebar
#    - Add the following secrets:
#      * GOOGLE_CLIENT_ID: Your Google's client ID
#      * GOOGLE_CLIENT_SECRET: Your Google's client secret.


# Use Your Google Cloud Project Client ID & Client Secrets (Using KTB GSC API v2)
CLIENT_ID = userdata.get("GOOGLE_CLIENT_ID")
CLIENT_SECRET = userdata.get("GOOGLE_CLIENT_SECRET")


if not CLIENT_ID:
  raise ValueError("GOOGLE_CLIENT_ID must be set")
if not CLIENT_SECRET:
  raise ValueError("GOOGLE_CLIENT_SECRET must be set")

In [ ]:
# This section handles authentication with google.
# It generates a URL that you have to use to get a code from google
# that you should paste here to finsih authentication.

# Define Oath scopes with read only access
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"

# Redirect URI to open Authorization Code Window in Browser
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

# Build URL that generates Authorization Code
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()

#Print Authorization URL
print("Go to the following link in your browser: " + authorize_url)

In [ ]:
auth_code = input("Enter your Authorization Code here:")
credentials = flow.step2_exchange(auth_code)

# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()

#authorize credentials
creds = credentials.authorize(http)

#building a service to access various features of GSC API
webmasters_service = build('searchconsole', 'v1', http=creds)

# List Your Sites

If you press play in this block, it will provide you with a list of the properties available to the Google Account you authenticated. I suggest that you copy and paste from this list, into the form in the next cell to ensure no errors.

In [ ]:
try:
    site_list = webmasters_service.sites().list().execute()
    sites =  site_list.get('siteEntry', [])
except HttpError as error:
    print(f"An error occurred: {error}")
    print(json.dumps(error.resp, indent=2))
    sites = ""

sorted_sites = sorted(sites, key=lambda x: x['siteUrl'])
print("\nSites you have access to (sorted alphabetically):")
for site in sorted_sites:
    print(f"- {site['siteUrl']}")

# GSC Data Extractor
---
1. Get started by entering the Site URL you want in the form below.
2. Run the code block. and
3. The output panel will ask you for a variety of inputs. You'll need to go through each of these in order to receive the output.
4. Once all inputs have completed, the code will iterate over each 25,000 rows of data until complete.
5. The script will output a CSV file based on your selection, in the file exporer.

In [ ]:
SITE_URL = "https://www.carricoseo.com/" # @param {"type":"string","placeholder":"url here"}

def get_gsc_data(start_date, end_date, dimensions, country=None, device=None, start_row=0, retries=3, backoff_factor=2):
    request = {
        'startDate': start_date.strftime("%Y-%m-%d"),
        'endDate': end_date.strftime("%Y-%m-%d"),
        'dimensions': dimensions,
        'rowLimit': 25000,
        'startRow': start_row,
        'aggregationType': 'byPage'
    }

    filters = []
    if country:
        filters.append({'dimension': 'country', 'operator': 'equals', 'expression': country.lower()})
    if device:
        filters.append({'dimension': 'device', 'operator': 'equals', 'expression': device.upper()})
    if filters:
        request['dimensionFilterGroups'] = [{'filters': filters}]

    for attempt in range(retries):
        try:
            response = webmasters_service.searchanalytics().query(siteUrl=SITE_URL, body=request).execute()
            rows = response.get('rows', [])
            return rows
        except HttpError as error:
            # Check if the error is retryable
            if error.resp.status in [500, 502, 503, 504]:
                print(f"Retryable error: {error}. Retrying in {backoff_factor ** attempt} seconds...")
                time.sleep(backoff_factor ** attempt)  # Exponential backoff
            else:
                print(f"An error occurred: {error}")
                print(json.dumps(error.resp, indent=2))
                return []
        except SSLEOFError as error:  # Handle SSLEOFError specifically
            print(f"Network error: {error}. Retrying in {backoff_factor ** attempt} seconds...")
            time.sleep(backoff_factor ** attempt)  # Exponential backoff
        except Exception as error:
            print(f"Unexpected error: {error}")
            return []

    print(f"Failed to fetch data after {retries} retries.")
    return []

# Function to recursively fetch all data
def fetch_all_data(start_date, end_date, dimensions_list, country=None, device=None, mode="separated"):
    all_combined_rows = []  # Collect all data if mode is combined

    for dimensions in dimensions_list:
        all_rows = []
        start_row = 0
        total_rows = 0

        print(f"\nStarting data pull for {dimensions}...")
        while True:
            rows = get_gsc_data(start_date, end_date, dimensions, country, device, start_row)
            if not rows:
                break
            all_rows.extend(rows)
            total_rows += len(rows)
            start_row += len(rows)

            print(f"Pulled {len(rows)} rows. Total rows pulled so far: {total_rows}")
            if len(rows) < 25000:
                break

        print(f"\nData pull complete for {dimensions}. Total rows pulled: {total_rows}")

        if mode == "separated":
            save_to_csv(all_rows, dimensions, mode)  # Save each separately
        else:
            all_combined_rows.extend(all_rows)  # Collect all rows for combined mode

    if mode == "combined":
        return all_combined_rows  # Return all data for saving outside the function
    return None  # Ensure no unintended return values


# Function to save data to CSV
def save_to_csv(rows, dimensions, mode, filename_prefix='gsc_data'):
    if not rows:
        print("\n *ISSUE*: No data to save.")
        return

    metrics = [metric for metric in rows[0].keys() if metric != 'keys']
    filename = f"{filename_prefix}_{'_'.join(dimensions)}.csv" if mode == "separated" else "gsc_data_combined.csv"

    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        header = dimensions + metrics
        csvwriter.writerow(header)
        for row in rows:
            row_data = row['keys'] + [row[metric] for metric in metrics]
            csvwriter.writerow(row_data)

    print(f"\nData saved to {filename}")

def get_date_range():
    """
    Prompts the user to select a date range.
    - 30 Days
    - 3 Months
    - 6 Months
    - 12 Months
    - 16 Months
    - Custom Date Range
    """

    prompt = """Select a Date Range Option:
1. 30 Days (from today minus 2 days)
2. 3 Months (from today minus 2 days)
3. 6 Months (from today minus 2 days)
4. 12 Months (from today minus 2 days)
5. 16 Months (from today minus 2 days)
6. Custom Date Range

Enter your choice (1, 2, 3, 4, 5, 6): """

    choice = input(prompt).strip()

    # Validate user input
    while choice not in ["1", "2", "3", "4", "5", "6"]:
        print("*ERROR*: Invalid choice, please ensure you select a valid option.")
        choice = input(prompt).strip()

    # Define the current date and adjust for the 2-day delay
    end = datetime.now() - timedelta(days=2)

    if choice == "1":
        # 30 Days
        start = end - timedelta(days=30)
    elif choice == "2":
        # 3 Months
        start = end - relativedelta(months=3)
    elif choice == "3":
        # 6 Months
        start = end - relativedelta(months=6)
    elif choice == "4":
        # 12 Months
        start = end - relativedelta(years=1)
    elif choice == "5":
        # 16 Months
        start = end - relativedelta(months=16)
    else:
        # Custom Date Range
        min_date = date.today() - timedelta(days=2) - relativedelta(months=16)
        max_date = date.today() - timedelta(days=2)

        # Set up date pickers and button.
        start_widget = widgets.DatePicker(description='Start date')
        end_widget = widgets.DatePicker(description='End date')

        # JavaScript to enforce min and max dates on the date pickers.
        script = Javascript(f"""
            const query = 'input[type=date]';
            document.querySelectorAll(query).forEach(function(inputElement) {{
                inputElement.setAttribute('min', '{min_date.strftime("%Y-%m-%d")}');
                inputElement.setAttribute('max', '{max_date.strftime("%Y-%m-%d")}');
            }});
        """)

        # Continue button to confirm date selection.
        button = widgets.Button(description="Confirm dates")
        button_clicked = False

        def on_click(b):
            nonlocal button_clicked
            button_clicked = True

        button.on_click(on_click)

        # Display widgets and script.
        display(start_widget, end_widget, button)
        display(script)

        # Wait for the user to click the confirm button.
        with ui_events() as poll:
            while button_clicked is False:
                poll(10)
                sleep(0.1)

        # Get start and end dates, ensuring start is earlier than end.
        start = start_widget.value
        end = end_widget.value

        if start > end:
            print("*ERROR*: Start date can't be later than end date. Swapping values.")
            start, end = end, start

    return start, end

def get_dimensions():
    print("Select Additional Dimensions: (query, page, date)")
    print("1. Predefined Set of Dimensions - [query], [query + date], [page + date]")
    print("2. Custom Dimensions (individually separated)")
    print("3. Custom Dimensions (combined)")

    choice = input("Enter choice (1,2,3): ")

    if choice == "1":
        dimensions = [["query"], ["query", "date"], ["page", "date"]]
        return dimensions, "separated"  # Change "preset" to "separated" so CSV files save correctly

    elif choice == "2" or choice == "3":
        custom_dimensions = input("Enter custom dimensions (comma-separated, e.g., 'query,page'): ").split(",")

        if choice == "2":
            return [[dim.strip()] for dim in custom_dimensions], "separated"  # Return dimensions and mode
        elif choice == "3":
            return [custom_dimensions], "combined"  # Return dimensions and mode

    else:
        print("Invalid choice. Defaulting to preset dimensions.")
        return [["query"], ["query", "date"], ["date", "page"]], "separated"  # Change "preset" to "separated"


print("\nPlease enter the following details:\n")
start_date, end_date = get_date_range()
country = input("Enter the 3 letter country code\(codes found here: https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3) (or leave blank to ignore): ")
country = country or None
device = input("Enter the device type (desktop, mobile, tablet or leave blank to ignore): ")
device = device or None
dimensions_list, mode = get_dimensions()  # Unpack the tuple into two variables

print("\nFetching data, please wait...")

# Fetch all data
all_data = fetch_all_data(start_date, end_date, dimensions_list, country, device, mode)

# Save combined data if applicable
if mode == "combined" and all_data:
    save_to_csv(all_data, dimensions_list[0], mode, "gsc_data_combined.csv")

# CarricoSEO HTML advertisement
html_content = """
<br><br>
<div style="border: 2px solid #C9A82D; background-color: #0C232A; padding: 15px; border-radius: 10px; width: 30%; margin-left: 0; text-align: left;">
  <div style="width: 100%; display: flex; align-items: center;">
    <span style="color: white; font-size: 16px; margin-right: 10px;">
      Built by
    </span>
    <a href="https://www.carricoseo.com/?utm_source=google_colab&utm_medium=referral&utm_campaign=colab_bannerad"
       target="_blank"
       style="text-decoration: none;">
      <img src="https://www.carricoseo.com/wp-content/uploads/2023/09/NEW-CarricoSEO-Logo-Light.svg"
           alt="CarricoSEO Logo"
           style="height: 25px; vertical-align: middle;">
    </a>
  </div>
  <br>
  <div style="margin-top: 10px; text-align: left; color: white; font-size: 14px;">
    This and many other one-click tools can be found at
    <a href="https://tools.carricoseo.com/?utm_source=google_colab&utm_medium=referral&utm_campaign=colab_tools"
       target="_blank"
       style="color: lightblue; text-decoration: none;">
      CS Tools.
    </a>
    I also have a blog full of other free resources, tools, and scripts
    <a href="https://www.carricoseo.com/resources/?utm_source=google_colab&utm_medium=referral&utm_campaign=colab_blog"
       target="_blank"
       style="color: lightblue; text-decoration: none;">
      found here!
    </a>
  </div>
</div>

"""
display(HTML(html_content))